# Big Data & BI – Optional Intro Notebook

**Topic:** Data, EDA


## Learning Objectives
By the end, you can:
- Download a real dataset from the open source projects.
- Evaluate a list of **KPIs**.
- Run simple analysis.


- Open "https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/GTNEJD"
- Download peoplesun_hh_anon.tab as csv (comma seperated valaues).
- Files named peoplesun_hh_odk_codebook.xlsx and peoplesun_hh_odk_choices.xlsx are also needed to decode the questions and answers.
- Place it in your data folder

## Key Performance Indicators (KPIs)

### KPI 1 — Electrification Rate (%)
**Definition:** Percentage of households with any access to electricity (grid or off-grid like solar/generator).  
**Formula:** Electrification Rate = (Households_with_electricity / Total_households) * 100  
**Purpose:** Measures extent of access.

---

### KPI 2 — Supply Reliability (Outages or Hours Without Power)
**Definition:** Average number of power outages per week OR hours without power per day among connected households.  
**Formula (example):** Avg_outages_per_week = SUM(outages_per_week) / N_connected_households  
**Purpose:** Captures quality/stability of supply.

---

### KPI 3 — Energy Burden (%)
**Definition:** Average share of monthly household spending (or income) devoted to electricity expenses.  
**Formula:** Energy_Burden = (Electricity_expenditure / Total_expenditure_or_income) * 100  
**Purpose:** Assesses affordability.


## 1) Setup & load

In [12]:
# --- setup ---
import pandas as pd
import numpy as np
import re
from pathlib import Path

DATA_PATH = Path("D:/SRH/big data/data folder/peoplesun_hh_anon.csv")  # students: place file here

# Helpful options
pd.set_option("display.max_colwidth", 120)
pd.set_option("display.max_columns", 120)

# --- load ---
# If the CSV has a different delimiter, change sep="," accordingly.
df = pd.read_csv(DATA_PATH, sep=",", low_memory=False)

print("Rows:", len(df))
df.head()


Rows: 3599


zone  state    eaid          lga                  urca_cat  \
0  North Central  Niger  NI_197      mashegu  <1hr to small city/town+   
1  North Central  Niger  NI_197      mashegu  <1hr to small city/town+   
2     North West   Kano  KN_104  garun_malam        <1hr to large city   
3     North West   Kano  KN_104  garun_malam        <1hr to large city   
4     North West   Kano  KN_104  garun_malam        <1hr to large city   

                                   hhid  q102  q103  q104  q105  q106  q107  \
0  d8af8ab5-30ab-4d9f-bfe4-81231dbe5dbf     1   NaN     1     1    36     2   
1  e8245d5c-8130-4e78-b4b0-1053b7ecbc9b     1   NaN     1     1    60     2   
2  435c8e27-517a-46b9-af04-48830e086d7a     1   NaN     1     1    45     1   
3  9303fa53-9fd2-41a9-9f0d-9567dbe5168e     1   NaN     1     1    47     1   
4  c62cc5a5-29c5-423b-9543-a7b05bda454b     1   NaN     1     1    45     1   

   q108  q109  q110  q201  q202  q203_1  q203_2  q203_3  q203_4  q204  q205  \
0   NaN   NaN   NaN     5     4      13       4       1       0     2     8   
1   NaN   NaN   NaN     6     2       4       3       2       1     3     9   
2   NaN   NaN   NaN     3     6       3       2       1       0     1     5   
3   NaN   NaN   NaN     3    15       7       2       1       0     1    10   
4   NaN   NaN   NaN     2     1       4       1       1       0     2     3   

   q206  q207_1  q207_2  q207_3  q208  q209  q210    q210_1    q211  q211__1  \
0     6       0       2       0    22    14    96  Politics    2 96        0   
1     6       0       0       0    14    13     1       NaN  1 3 96        1   
2     6       1       1       0    14     8     2       NaN     2 8        0   
3     6       3       1       0    14     8     2       NaN     2 7        0   
4     6       0       1       0    24    24     2       NaN       2        0   

   q211__2  q211__3  q211__4  q211__5  q211__6  q211__7  q211__8  q211__9  \
0        1        0        0        0        0        0        0        0   
1        0        1        0        0        0        0        0        0   
2        1        0        0        0        0        0        1        0   
3        1        0        0        0        0        1        0        0   
4        1        0        0        0        0        0        0        0   

   q211__10  q211__11  q211__12  q211__0  q211__96  \
0         0         0         0        0         1   
1         0         0         0        0         1   
2         0         0         0        0         0   
3         0         0         0        0         0   
4         0         0         0        0         0   

                                 q211_1  q212  q212_1  q213 q213_1  q213_1__1  \
0  Politics and farming for consumption     1     1.0     1      1        1.0   
1               Farming for consumption     0     NaN     1      1        1.0   
2                                   NaN     1     1.0     1      1        1.0   
3                                   NaN     1     1.0     0    NaN        NaN   
4                                   NaN     0     NaN     0    NaN        NaN   

   q213_1__2  q213_1__96  q214  q214__1  q214__2  q214__3  q214__4  q214__5  \
0        0.0         0.0  1 11        1        0        0        0        0   
1        0.0         0.0    11        0        0        0        0        0   
2        0.0         0.0    11        0        0        0        0        0   
3        NaN         NaN    11        0        0        0        0        0   
4        NaN         NaN    11        0        0        0        0        0   

   ...  q402__4  q402__5  q402__6    q404  q404__1  q404__2  q404__3  q404__4  \
0  ...        1        0        1    2 11        0        1        0        0   
1  ...        1        0        1       0        0        0        0        0   
2  ...        1        0        1      11        0        0        0        0   
3  ...        1        1        1  2 11 1        1        1   

## 2) Quick EDA (Exploratory Data Analysis)

In [13]:
print(df.shape)
print(df.head())

(3599, 293)
            zone  state    eaid          lga                  urca_cat  \
0  North Central  Niger  NI_197      mashegu  <1hr to small city/town+   
1  North Central  Niger  NI_197      mashegu  <1hr to small city/town+   
2     North West   Kano  KN_104  garun_malam        <1hr to large city   
3     North West   Kano  KN_104  garun_malam        <1hr to large city   
4     North West   Kano  KN_104  garun_malam        <1hr to large city   

                                   hhid  q102  q103  q104  q105  q106  q107  \
0  d8af8ab5-30ab-4d9f-bfe4-81231dbe5dbf     1   NaN     1     1    36     2   
1  e8245d5c-8130-4e78-b4b0-1053b7ecbc9b     1   NaN     1     1    60     2   
2  435c8e27-517a-46b9-af04-48830e086d7a     1   NaN     1     1    45     1   
3  9303fa53-9fd2-41a9-9f0d-9567dbe5168e     1   NaN     1     1    47     1   
4  c62cc5a5-29c5-423b-9543-a7b05bda454b     1   NaN     1     1    45     1   

   q108  q109  q110  q201  q202  q203_1  q203_2  q203_3  q203_4  q20

## 3) KPI 1 — Electrification Rate (%)

In [14]:
# --- explore ---

## 4) KPI 2 — Reliability (choose the best available)

In [15]:
# --- explore ---

## 5) KPI 3 — Energy Burden (% of total monthly expenditure or income)

In [16]:
# --- explore ---